In [1]:
import torch
from jackpot import Jackpot

params = {
    "experiment_name": "jmlr_blind_inverse_problem",
    "save_plot": True,
    "n_singular_pairs": 5,
    "max_compute_time": 3600,
    "load_sing_pairs": False,
    "save_sing_pairs": True,
    "D": 1,
    "epsilon": 1e-1,
    "n_points_per_axis": 101,
    "grid_length": 1e-4,
    "load_manifold": False,
    "save_manifold": True,
}

# ------------------------------------
#  DIRECT MODEL 
# ------------------------------------
import torch.optim as optim
import torch

import matplotlib.pyplot as plt
from pathlib import Path

import deepinv as dinv

######## IMPORT THE MODEL #################################
from article_examples.blind_inv_pb.deblur_simple import Deblurring

import imageio.v3 as iio

from tqdm import tqdm
import os

# Device & dtype setup
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64
factory_kwargs = {"device": device, "dtype": dtype}
torch.manual_seed(0)

# Example: Blind inverse problem
img_size = 100 # 400
image_filename = "blob3.jpg"
max_iter_DPIR = 10
i_max_zernike = 12
psf_size = 51

num_workers = 8
batch_size = 1

PATH = Path("./dataset/")
image_name = image_filename[:-4]
img_filename = PATH / Path(image_filename)

def load_image(img_name, img_size = None):
    im = iio.imread(img_name)
    if len(im.shape) == 2:
        im = im[:,:,None]
    
    im = (torch.tensor(im, device = device, dtype = dtype)[:,:,:3] *  
            torch.ones((1,1,3), device = device, dtype = dtype)) /255
    im = im.permute((2,0,1))[None,...]
    
    if img_size != None:
        im = im[:,:,:img_size,:img_size]
    
    return im

x = load_image(img_filename, img_size = img_size)

deblur_model = Deblurring(device = device, dtype = dtype)
deblur_model.set_variables(img_size = img_size, batch_size = batch_size,
                           num_workers = num_workers, max_iter_DPIR = max_iter_DPIR,
                           i_max_zernike = i_max_zernike, psf_size = psf_size)

x, theta_star, y = deblur_model.generate_random_coeffs(x = x)

sigma_noise = deblur_model.noise_level_img
deblur_model.set_model(y)
Phi = deblur_model.Phi
x_est = theta_star
# ------------------------------------

/home/munier/python_envs/jackpot_env/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------------------------------
# JACKPOT INITIALIZATION
# ------------------------------------
jack = Jackpot(Phi, x_est)

jack.set_params(**params)
# ------------------------------------


# ------------------------------------
# JACOBIAN SINGULAR SPECTRUM
# ------------------------------------
jack.jacobian_spectrum()

# Plot spectrum
jack.jacobian_spectrum_plot()
# ------------------------------------


# ------------------------------------
# JACKPOT MANIFOLD
# ------------------------------------
jack.manifold()

# Plot discrepancy
jack.plot_discrepancy(
    in_SNR=True,
    levels=[20, 30, 40],
    color_levels=["g", "b", "r"],
)
# ------------------------------------

RuntimeError: cusolver error: CUSOLVER_STATUS_INTERNAL_ERROR, when calling `cusolverDnCreate(handle)`. If you keep seeing this error, you may use `torch.backends.cuda.preferred_linalg_library()` to try linear algebra operators with other supported backends. See https://pytorch.org/docs/stable/backends.html#torch.backends.cuda.preferred_linalg_library

In [3]:
jack.mani_dataset_evals.shape

AttributeError: 'NoneType' object has no attribute 'shape'